# Workbook for optimizing foci segmentation thresholds

This workbook provides step-by-step instructions for determining segmentation thresholds for identifying foci within cells. __Do not edit this workbook file directly!__ Make a copy for each optimization that you perform and edit that version. Save this template in an unedited form for future experiments.

This workbook uses a built-in "optimization mode" that only pulls the first three images out of multi-image .czi files to dramatically speed up processing. This will allow you to tinker with thresholds and find the ideal cutoffs.

I found that the most effective way to define these cutoffs was to use a positive control image - one where there were many foci - and a negative control image with few to no foci to set the thresholds. This allows you to identify cutoffs that simultaneously maximize sensitivity while minimizing false positives.

__Before running this notebook, you must have completed the setup notebook in the install folder.__

In [7]:
# import required code
import sys
sys.path.append('/Users/nweir/Dropbox/code/csth-imaging')
from csth_analysis import find_cells, segment_cells, foci
import os
from skimage import io
# uncomment the following lines if you have jupyternotify installed and want to use it
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

The cell below defines the file paths for the files you will need during this process. Change each variable as needed.

In [2]:
dATG7_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dAtg7_3hrTor_4Pos_2_AiryscanProcessing.czi'
dATG7_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dAtg7_3hrTor_Empty_AiryscanProcessing.czi'
dFIP200_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dFIP200_3hrTor_9Pos_3_AiryscanProcessing.czi'
dFIP200_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dFIP200_3hrTor_Empty_AiryscanProcessing.czi'
WT_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_WT_3hrTor_20Pos_1_AiryscanProcessing.czi'
WT_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_WT_3hrTor_Empty_AiryscanProcessing.czi'
dTMEM_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dTMEM_3hrTor_10Pos_1_AiryscanProcessing.czi'
dTMEM_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dTMEM_3hrTor_Empty_AiryscanProcessing.czi'
dVPS37A_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dVPS_3hrTor_10Pos_2_AiryscanProcessing.czi'
dVPS37A_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/LC3-WIPI_HEK_dVPS_3hrTor_Empty_AiryscanProcessing.czi'
svm_pkl = '/Users/nweir/Dropbox/code/csth-imaging/trained_svm.pkl' # change /path/to/ so it points to the csth-imaging folder

Now that you have defined the important variables, you will perform pre-processing. This takes the following steps:
1. Load images
2. Identify regions of the field containing cells
3. Identify nuclei
4. Segment cells  
    _Note:_ By default this segments cells using the 488 wavelength. Change 488 to your desired wavelength in the splitter.segment_cells() command to change wavelengths.
5. Initialize foci detection object.

The steps are indicated in the code blocks.

In [3]:
%%notify
## NOTE: RUNNING THIS CELL WILL TAKE A WHILE! DON'T BE CONCERNED! ##

# 1. load images and the trained SVM focus classifier, and 2.
dVPS37A_finder = find_cells.MultiFinder(filename=dVPS37A_czi,
                                         bg_filename=dVPS37A_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dATG7_finder = find_cells.MultiFinder(filename=dATG7_czi,
                                         bg_filename=dATG7_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dTMEM_finder = find_cells.MultiFinder(filename=dTMEM_czi,
                                         bg_filename=dTMEM_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dFIP200_finder = find_cells.MultiFinder(filename=dFIP200_czi,
                                         bg_filename=dFIP200_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
WT_finder = find_cells.MultiFinder(filename=WT_czi,
                                         bg_filename=WT_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')

# 2. find cells
WT_splitter = segment_cells.CellSplitter(WT_finder)
dVPS37A_splitter = segment_cells.CellSplitter(dVPS37A_finder)
dATG7_splitter = segment_cells.CellSplitter(dATG7_finder)
dTMEM_splitter = segment_cells.CellSplitter(dTMEM_finder)
dFIP200_splitter = segment_cells.CellSplitter(dFIP200_finder)

# 3. identify nuclei
WT_splitter.segment_nuclei(verbose=True)
dVPS37A_splitter.segment_nuclei(verbose=True)
dATG7_splitter.segment_nuclei(verbose=True)
dTMEM_splitter.segment_nuclei(verbose=True)
dFIP200_splitter.segment_nuclei(verbose=True)

# 4. Segment cells.
WT_splitter.segment_cells(488, verbose=True)
dVPS37A_splitter.segment_cells(488, verbose=True)
dATG7_splitter.segment_cells(488, verbose=True)
dTMEM_splitter.segment_cells(488, verbose=True)
dFIP200_splitter.segment_cells(488, verbose=True)

# 5. Initialize foci detection objects.
dVPS37A_foci = foci.Foci(dVPS37A_splitter, verbose=True)
dATG7_foci = foci.Foci(dATG7_splitter, verbose=True)
dFIP200_foci = foci.Foci(dFIP200_splitter, verbose=True)
dTMEM_foci = foci.Foci(dTMEM_splitter, verbose=True)
WT_foci = foci.Foci(WT_splitter, verbose=True)

Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
MultiFinder created.
generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1

generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
correcting slice labels...
One continuous stretch defined as in focus.
appending outputs...
mask #3 complete.

generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for

thresholding floor set to 2300
slice-by-slice cutoffs:
[  131.5   116.5   126.    141.    138.5   167.    184.5   214.    273.5
   355.    431.5   579.5   800.5  1320.5  2036.   2855.5  3607.5  4123.5
  4459.   4725.   4871.   4968.   5187.   5418.5  5565.   5673.   5752.
  5741.5  5683.   5616.5  5550.5  5681.5  5642.   5477.   5308.5  5025.5
  4561.   4148.   3558.5  2857.   2163.   1663.5  1339.5  1118.    898.
   713.    572.5   465.5   385.    333.5   326.5   319.    300.    261.5
   249.5   227.    229.5   197.    179. ]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

removing small perinuclear foci...
2 raw segmented DAPI foci
object volumes in pixels:
[404484 433377]
removing objec

input image has 4 dimensions (multiple imgs)
segmenting image #1 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 5150
slice-by-slice cutoffs:
[   126.     101.     113.     140.     148.5    149.     170.5    177.5
    207.5    206.5    227.5    285.5    345.5    418.5    479.     521.5
    569.5    603.     776.5   1256.    2304.    3841.5   5533.5   7376.
   9255.   10886.   11879.   12251.5  12239.   11862.5  11923.   11962.5
  12078.5  12209.   12295.   12285.   12131.5  11951.   11821.   11664.5
  11724.5  12112.5  12484.   12763.   12876.5  12779.   12501.5  12119.
  11638.5  11128.5  11507.5  11596.   11531.5  11983.   11933.   11687.5
  11415.5  11391.5  11200. ]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentO

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell mask means:
[1017.4359255125961, 1054.4347881077417, 1091.0862903137199, 1125.3433471582407, 1156.0176574172119, 1182.7457347666307, 1205.7891162687101, 1225.5370842503478, 1242.6312716797104, 1257.7860864450643]
slopes:
[36.825182400561914, 35.45427952524949, 32.465683551745997, 28.701193804194986, 24.885729425749105, 21.395674741858556, 18.421077705500124, 16.124501097358234]
slope deltas:
[ 1.37090288  2.98859597  3.76448975  3.81546438  3.49005468  2.97459704
  2.29657661]
desired erosions: 5
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[1151.1084545607657, 1198.7440820678776, 1246.2293535631213, 1290.9057918198373, 1331.0252486446216, 1365.9849303392489, 1396.1076246527966, 1422.1993064825638, 1445.0281003589896, 1465.347150691073]
slopes:
[47.5604495011778, 46.080854875979867, 42.397947540750124, 37.539569259705786, 32.54118800408753, 28.107188071657447, 24.460237853096487, 21.573922104254621]
slope deltas:
[ 1.47959463  3.68290734  4.85837828  4.99838126 

removing small foci...
2 raw segmented cells
0 foci volume < 1000 px, removing...

segmenting cells in image #3 out of 3...
performing Euclidean distance xform...
smoothing the distance map...
generating segmentation seeds from nuclei_centers...
watershed segmenting cells from nuclei centers...
segmentation complete for image #3.
removing cells that contact the edge of the image.
removing small foci...
2 raw segmented cells
0 foci volume < 1000 px, removing...

checking that nuclei have already been segmented...
eroding cell edges...
eroding mask 0...
cell mask means:
[1560.0276340747159, 1611.8674532466871, 1662.5402410764621, 1710.1568435389381, 1753.7957175904928, 1793.511634471377, 1829.6644425158142, 1862.8408389073982, 1893.5715548799999, 1922.1340379649373]
slopes:
[51.256303500873059, 49.144695146125514, 45.62773825701538, 41.677395466219423, 37.934362462660715, 34.6646022180106, 31.953556182092825, 29.64659952876957]
slope deltas:
[ 2.11160835  3.51695689  3.95034279  3.743033

<IPython.core.display.Javascript object>

Next, you're ready to try segmenting foci and checking the output to see if it looks correct.

A bit of background on how Canny edge detection works so you can logically select values. There are two threshold numbers that go into detecting foci:
- high threshold: the minimum edge intensity to _seed_ a new object. The entire object doesn't have to have an edge this sharp, but if no part of it does, it won't be identified.
- low threshold: the minimum edge intensity to _grow_ an object. If one point already satisfied the high threshold, it can grow even if it has a dimmer edge as long as that dimmer edge is above the low threshold.  

If this doesn't make sense, read about the Canny edge detection algorithm on Wikipedia or elsewhere.

I often start with values of 10,000 and 7,500 for the high and low thresholds respectively, so those are the default values below.

Running the code block below will perform segmentation and save the raw images as well as the segmented foci to the output_dir you select in the first line. You can then open these images in fiji, overlay the segmented foci onto the raw images, and determine if the threshold is too high/too low. The code will also output some descriptive information about the number of dots, their intensities, etc. which may be useful to you in deciding thresholds.

In [5]:
%%notify
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/outputs/'  # change this to where you'd like the output images to go.
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# save raw images
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dATG7_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dAtg7_raw_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.imgs[c][im, :, :, :])
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dVPS37A_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_raw_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.imgs[c][im, :, :, :])    
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dTMEM_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_raw_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.imgs[c][im, :, :, :])
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dFIP200_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_raw_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.imgs[c][im, :, :, :])
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, WT_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'WT_raw_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.imgs[c][im, :, :, :])

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/outputs/dAtg7_raw_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/outputs/dAtg7_raw_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/outputs/dAtg7_raw_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/outputs/dAtg7_raw_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/s

## The following cell is for running segmentation on both channels.

In [8]:
%%notify
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_wipi_timecourse/outputs/'
# edit the (high, low) threshold pairs below to optimize segmentation.
green_thresholds = (11000, 8250) 
red_thresholds = (15000, 11250)
# perform segmentation
dATG7_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dATG7_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dATG7_foci_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.foci[c][im])
dVPS37A_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: (7500, 6000)})
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dVPS37A_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_foci_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.foci[c][im])
dTMEM_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dTMEM_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_foci_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.foci[c][im])
dFIP200_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dFIP200_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_foci_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.foci[c][im])
WT_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(WT_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'WT_foci_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.foci[c][im])

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (11000, 8250)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 1249.2028314
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 101 foci in image
cell mean: 1027.52036119
cell standard deviation: 1085.52182591
intensity cutoff for removal: 5912.36857779
-----filtering foci-----
focus intensity: 69.385472251
removing focus
focus intensity: 4586.45

[163]
matched to one parent cell.
current ID: 65
  overlapping parent cell(s): 
[]
  # of pixels overlapping with parent(s):
[]
[]
[]
current ID: 67
  overlapping parent cell(s): 
[1 2]
  # of pixels overlapping with parent(s):
[  6 289]
  more than one overlapping cell.
  determining which cell overlaps more.
  parent cell: 2
current ID: 73
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[215]
matched to one parent cell.
current ID: 74
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[468]
matched to one parent cell.
current ID: 76
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[45]
matched to one parent cell.
current ID: 79
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[80]
matched to one parent cell.
current ID: 84
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[226]
matched to one parent cell.
current ID: 85
  overlapping parent cell(s)

focus intensity: 5425.0
removing focus
focus intensity: 4770.68421053
removing focus
focus intensity: 6073.8
focus intensity: 5377.68421053
removing focus
focus intensity: 6901.63636364
focus intensity: 8266.31111111
focus intensity: 7689.69411765
focus intensity: 4980.0952381
removing focus
focus intensity: 7720.91525424
focus intensity: 8754.65921788
focus intensity: 8503.07086614
focus intensity: 7439.20325203
focus intensity: 4225.95833333
removing focus
focus intensity: 4672.6
removing focus
focus intensity: 5522.45833333
removing focus
focus intensity: 4771.33333333
removing focus
focus intensity: 9107.23529412
focus intensity: 10924.7355491
focus intensity: 14702.6923077
focus intensity: 5040.8
removing focus
focus intensity: 5399.8
removing focus
focus intensity: 6206.4
focus intensity: 5839.12903226
after eliminating dim foci: 76 foci in image
eliminating foci that reside outside of cells...
eliminating intranuclear foci...
74 final foci
--matching foci to parent cells--
curre

[2]
  # of pixels overlapping with parent(s):
[93]
matched to one parent cell.
current ID: 113
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[168]
matched to one parent cell.
current ID: 115
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[30]
matched to one parent cell.
current ID: 116
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[41]
matched to one parent cell.
current ID: 118
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[43]
matched to one parent cell.
current ID: 124
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[40]
matched to one parent cell.
current ID: 126
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[44]
matched to one parent cell.
current ID: 127
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[90]
matched to one parent cell.
current ID: 128
  overlapping parent c

current ID: 39
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[121]
matched to one parent cell.
current ID: 40
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[62]
matched to one parent cell.
current ID: 42
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[83]
matched to one parent cell.
current ID: 45
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[231]
matched to one parent cell.
current ID: 46
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[320]
matched to one parent cell.
current ID: 48
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[66]
matched to one parent cell.
current ID: 50
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[91]
matched to one parent cell.
current ID: 51
  overlapping parent cell(s): 
[1 2]
  # of pixels overlapping with parent(s):
[ 90 117]
  more than one ov

...

focus intensity: 6227.8
removing focus
focus intensity: 12897.0833333
removing focus
focus intensity: 11694.0982143
removing focus
focus intensity: 13121.9212691
removing focus
focus intensity: 16422.4090909
focus intensity: 20284.5021645
focus intensity: 14258.7885117
removing focus
focus intensity: 14056.7853143
removing focus
focus intensity: 13506.028436
removing focus
focus intensity: 31641.7119565
focus intensity: 10924.555794
removing focus
focus intensity: 12109.7241379
removing focus
focus intensity: 9238.56122449
removing focus
focus intensity: 13296.0328638
removing focus
focus intensity: 6806.09333333
removing focus
focus intensity: 17961.7062937
focus intensity: 16807.6798246
focus intensity: 11083.7469136
removing focus
focus intensity: 6071.26923077
removing focus
focus intensity: 14886.4326923
focus intensity: 16134.9420732
focus intensity: 14461.0510204
focus intensity: 10198.3658537
removing focus
focus intensity: 8330.87632509
removing focus
focus intensity: 12099.95

focus intensity: 9238.36774194
removing focus
focus intensity: 8646.24137931
removing focus
focus intensity: 10890.5675676
removing focus
focus intensity: 9758.06779661
removing focus
focus intensity: 4669.4
removing focus
focus intensity: 11053.4516129
removing focus
focus intensity: 10588.8536585
removing focus
focus intensity: 10972.1945946
removing focus
focus intensity: 12006.5868307
removing focus
focus intensity: 10883.0694444
removing focus
focus intensity: 4258.2
removing focus
focus intensity: 10786.5659824
removing focus
focus intensity: 17829.2857143
focus intensity: 13717.3213213
removing focus
focus intensity: 12555.8986486
removing focus
focus intensity: 6876.86206897
removing focus
focus intensity: 11501.603241
removing focus
focus intensity: 14540.9279279
focus intensity: 11880.6528497
removing focus
focus intensity: 10498.7152146
removing focus
focus intensity: 8925.33333333
removing focus
focus intensity: 8503.50297422
removing focus
focus intensity: 9711.56140351
re

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

Edit the thresholds in the cell above and re-run as needed until you reach ideal thresholds.

## The following cell is for running segmentation on just one channel.

In [ ]:
desired_channel = 488  # change this to 488 for green or 561 for red
# edit the (high, low) threshold pair below to optimize segmentation.
thresholds = (10000, 7500) 
# perform segmentation
neg_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
pos_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
# save segmentation outputs
for im in range(0, len(pos_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'pos_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             pos_foci.foci[desired_channel][im])
for im in range(0, len(neg_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'neg_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             neg_foci.foci[desired_channel][im])